In [1]:
# --- To be able to import from other modules here ---
import sys; import os; sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
from google import genai
from google.genai import types
from utils.save_response import writeMd
from dotenv import load_dotenv
from pydantic import BaseModel, create_model, Field

### Loading the environment variables
We can use the dotenv library to load all the environment variables written in a local.env file.
 --> We only have the `GEMINI_API_KEY` variable ...

In [3]:
env_path = "../local.env"
load_dotenv(dotenv_path=env_path)

True

### Google genai
You can refer to the [official documentation](https://ai.google.dev/gemini-api/docs) for more details on how to use the Google Gemini API with Python.

First, have a look at the [Quickstart](https://ai.google.dev/gemini-api/docs/quickstart) on the left panel. Note that you don't need to install anything (ignore pip install instructions), as we have already installed everything for you. 

Try out the code with different prompt contents and different models to see if your environment works!

![fig1](figs/gemini_quickstart.png)


In [4]:
# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

### TEXT GENERATION WITH THINKING
Here, you can see all the different knobs that you can control when calling a model for text generation.

You can choose whether or not to `include_thoughts` by setting it to True/False, or change the maximum number of thinking tokens as the `thinking_budget`.

You can change how probabilistic/deterministic the model samples tokens at its output: `temperature` of 0.0 means complete deterministic here ...

You can also control the maximum number of output tokens ...

For a more comprehensive list and explanation of variables, you can read [Generating content](https://ai.google.dev/api/generate-content#v1beta.GenerationConfig) page.

In the following cell, you see an example of using `gemini-2.5-flash` as a math assistant.

We have provided a *writeMd* function for you (which is in the *utils* module) that saves outputs (along with gemini's thinking summary) in a markdown file.

The *writeMd* saves the output in the `no_backup/markdown_files` folder, you can open the generated file and use `Ctrl+Shift+V` to show it in preview mode ...

In [5]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=10_000, include_thoughts=True), # Disables thinking
        temperature=0.0,  # Sets deterministic output
        max_output_tokens=20_000, # Sets maximum output tokens
        seed=42,  # Sets a seed for reproducibility
        candidate_count=1,  # Number of response candidates to generate
        system_instruction="You're a math teacher and assistant that provides guided answers to math questions.",
    ),
    contents="What is the Euler's formula? How did he find it out?", # The prompt to generate content for
)

writeMd(response, filename="output.md")



file:///users/micas/mahmadza/Documents/THESIS/CAD/no_backup/markdown_files/output.md




### STRUCTURED OUTPUT

We can use the **pydantic** library to define an output style (or *response schema*) that the LLM should stick to:

1. Using the `BaseModel`:
    create a class that inherits from *BaseModel* and write its fields with their corresponding data types and explanation

2. Using the `create_model`:
    create a dictionary of the field names and their corresponding data types and use the *create_model* to create a data class from it

Read the **Structured output** tab in the left panel of Gemini api documentation for more information. [click here](https://ai.google.dev/gemini-api/docs/structured-output)

In [34]:
# 1.
class CityInfo_1(BaseModel):
    city: str = Field(..., description="Name of the city")
    population: int = Field(..., description="Population of the city")
    area: float = Field(..., description="Area of the city in square kilometers")

# 2.
fields_dict = {'city': str, 'population': int, 'area': float}
CityInfo_2 = create_model(
    'CityInfo_2',
    **{field_name: (data_type, Field(...)) for field_name, data_type in fields_dict.items()}
)


# The Field can also include other metadata like ge/le (greater or equal, less or equal) ...

# 3.
class CityInfo_3(BaseModel):
    city: str = Field(..., description="Name of the city")
    population: int = Field(..., description="Population of the city", ge=10_000, le=50_000_000)
    area: float = Field(..., description="Area of the city in square kilometers", ge=0.5, le=500_000)


response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Provide information about city of Tokyo including its population and surface area.",
    config={
        "response_mime_type": "application/json",
        "response_schema": CityInfo_3
    },
)
print(response.text)
response_data = response.parsed
response_dict = dict(response_data)

{"city":"Tokyo","population":14000000,"area":2194}


In [36]:
print(type(response.text))  # str
print(type(response_data))  # CityInfo
print(type(response_dict))  # dict

<class 'str'>
<class '__main__.CityInfo_3'>
<class 'dict'>


### FUNCTION CALLING: Equipt the LLM with a Calculator!

You can write your own python function and write a decleration for it (see below cell) and give it to the LLM as a **tool**.

The LLM will decide to call it when it sees necessary.

The decleration of the function should include all its argument descriptions.

When the LLM decides to call the function, it will provide the name of the function as well as the arguments to pass to the function. **It will not actually run the function by itself**.

Our python code should be written in a way that detects when the LLM contains a function calling part, runs the function, and gives back the output of the function to the LLM as a second call.

The reslut of the function should be appended to the contents (ongoing conversation from previous call) so that the LLM knows this second call is the continuation of the first call and contains the result of the function run. 

For more information, please visit the [Function Calling](https://ai.google.dev/gemini-api/docs/function-calling) tab in the gemini api documentation.

In the below example, our function (external tool) is a calculator! However, in your project, the RL-Sizer will be the external optimizer tool.

Your system prompt to the LLM can explain when to call the tools. This is not necessary for easy tools like a calculator;
    
 but for sophisticated tools like the RL sizer, you can indicate when it should be used, for how many runs, etc. 

In [ ]:
# Define a function that the model can call to calculate basic arithmetic operations
calculator_declaration = {
    "name": "calculator",
    "description": "A simple calculator to perform basic arithmetic operations.",
    "parameters": {
        "type": "object",
        "properties": {
            "operation": {
                "type": "string",
                "description": "The arithmetic operation to perform: add, subtract, multiply, divide, power, \
                                                                     exponent, square_root, log10, ln, to_degrees, \
                                                                     to_radians, sine, cosine, tangent, arcsine, \
                                                                     arccosine, arctangent.",
            },
            "num1": {
                "type": "number",
                "description": "The first number.",
            },
            "num2": {
                "type": "number",
                "description": "The second number (if needed).",
            },
        },
        "required": ["operation", "num1"],
    },
}

import math
# This is the actual function that would be called based on the model's suggestion
def calculator(operation: str, num1: float, num2: float = None) -> float:
    if operation == "add":
        return num1 + num2
    elif operation == "subtract":
        return num1 - num2
    elif operation == "multiply":
        return num1 * num2
    elif operation == "divide":
        if num2 != 0:
            return num1 / num2
        else:
            return "Error: Division by zero"
    elif operation == "power":
        return num1 ** num2
    elif operation == "exponent":
        return math.exp(num1)
    elif operation == "square_root":
        return math.sqrt(num1)
    elif operation == "log10":
        return math.log10(num1)
    elif operation == "ln":
        return math.log(num1)
    elif operation == "to_degrees":
        return math.degrees(num1)
    elif operation == "to_radians":
        return math.radians(num1)
    elif operation == "sine":
        return math.sin(num1)
    elif operation == "cosine":
        return math.cos(num1)
    elif operation == "tangent":
        if num1 % (math.pi/2) == 0 and (num1 / (math.pi/2)) % 2 != 0:
            return "Error: Tangent undefined at this angle"
        return math.tan(num1)
    elif operation == "arcsine":
        return math.asin(num1)
    elif operation == "arccosine":
        return math.acos(num1)
    elif operation == "arctangent":
        return math.atan(num1)
    else:
        return "Error: Unknown operation"

tools = types.Tool(function_declarations=[calculator_declaration])
config = types.GenerateContentConfig(tools=[tools])


In [67]:
prompt_1 = "Calculate the multiplication of 15.5 and 24.3."
prompt_2 = "What is the square root of 256?"
prompt_3 = "Compute the log10 of 1000."
prompt_3_= "give me the natural logarithm of 1000."
prompt_4 = "Add 100 and 250."
prompt_5 = "Divide 500 by 0."
prompt_6 = "Raise 2 to the power of 8."
prompt_7 = "What is the sine of 3*pi/2?"
prompt_8 = "What is the tangent of 90 degrees?"

# Define user prompt
contents = [
    types.Content(
        role="user", parts=[types.Part(text=prompt_7)]
    )
]

# Send request with function declarations
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=contents,
    config=config,
)

print("="*100)
print(response.candidates[0].content.parts[0].function_call)

# Extract tool call details, it may not be in the first part.
tool_call = response.candidates[0].content.parts[0].function_call

if tool_call.name == "calculator":
    result = calculator(**tool_call.args)
    print("="*100)
    print(f"Function execution result:\n {result}")

# Create a function response part
function_response_part = types.Part.from_function_response(
    name=tool_call.name,
    response={"result": result},
)

# Append function call and result of the function execution to contents
contents.append(response.candidates[0].content) # Append the content from the model's response.
contents.append(types.Content(role="user", parts=[function_response_part])) # Append the function response

client = genai.Client()
final_response = client.models.generate_content(
    model="gemini-2.5-flash",
    config=config,
    contents=contents,
)
print("="*100)
print(final_response.text)


if final_response.candidates[0].content.parts[0].function_call:
    print("another function call detected!")

id=None args={'num1': 4.71238898025, 'operation': 'sine'} name='calculator'
Function execution result:
 -1.0
The sine of 3π/2 is -1.


### ReAct agent: Multiple calls to the Tool

#### ReAct --> Reason & Act

As you can see in the above example, when we use prompt 8, the LLM tries to first call the calculator for converting degrees to radians.

After that, it will try to use the calculator for a second time for the tangent of the converted angle ...

Therefore, our code can have a loop: while the output of the LLM still contains a call to a function, our code should handle the function calling and appending the result to the contents ....

In [98]:
prompt = input("Enter your own prompt: ")
if prompt.strip() == "":
    prompt = "12 ^ (tangent(135 degrees) - exp(2))"

print ("user prompt:", prompt)

# Define user prompt
contents = [
    types.Content(
        role="user", parts=[types.Part(text=prompt)]
    )
]

# Send request with function declarations
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=contents,
    config=config,
)

# Extract tool call details, it may not be in the first part.
tool_call = response.candidates[0].content.parts[0].function_call

while tool_call:
    if tool_call.name == "calculator":
        result = calculator(**tool_call.args)
        print("="*100)
        print(f"Function execution result:\n {result}")
    # Create a function response part
    function_response_part = types.Part.from_function_response(
        name=tool_call.name,
        response={"result": result},
    )

    # Append function call and result of the function execution to contents
    contents.append(response.candidates[0].content) # Append the content from the model's response.
    contents.append(types.Content(role="user", parts=[function_response_part])) # Append the function response

    client = genai.Client()
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        config=config,
        contents=contents,
    )
    tool_call = response.candidates[0].content.parts[0].function_call

md_string = ""
# monitoring the thinking summary and the final answer
for part in response.candidates[0].content.parts:
    if not part.text:
        continue
    if part.thought:
        md_string += f"# Thought Summary\n{part.text}\n"
    else:
        md_string += f"# Answer Text\n{part.text}\n"

print("="*100)
print(md_string)

user prompt: 79247 * 2343 - 44356/tangent(44 degrees)
Function execution result:
 0.017704699278685777
Function execution result:
 2505323.5472572544
Function execution result:
 185675721
Function execution result:
 183170397.45274276
# Answer Text
The answer is 183170397.45274276



What you have above is a POWERFUL calculator that can handle any complex type of calculation: be it written in sentence form or mathematical expression form 😉